# setup

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%%capture
!pip install git+https://github.com/EleutherAI/sae.git

In [3]:
# you should load this before cloning repo files
# from .config import SaeConfig
# from .utils import decoder_impl

from sae.config import SaeConfig
from sae.utils import decoder_impl
from sae import Sae

Triton not installed, using eager implementation of SAE decoder.


In [4]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import json
from fnmatch import fnmatch
from pathlib import Path
from typing import NamedTuple, Optional, Callable, Union, List, Tuple
# from jaxtyping import Float, Int

import einops
import torch
from torch import Tensor, nn
from huggingface_hub import snapshot_download
from natsort import natsorted
from safetensors.torch import load_model, save_model

device = "cuda" if torch.cuda.is_available() else "cpu"

In [51]:
from collections import Counter
import pandas as pd
from IPython.display import display

## corr fns

In [6]:
def batched_correlation(reshaped_activations_A, reshaped_activations_B, batch_size=100):
    # Ensure tensors are on GPU
    if torch.cuda.is_available():
        reshaped_activations_A = reshaped_activations_A.to('cuda')
        reshaped_activations_B = reshaped_activations_B.to('cuda')

    # Normalize columns of A
    mean_A = reshaped_activations_A.mean(dim=0, keepdim=True)
    std_A = reshaped_activations_A.std(dim=0, keepdim=True)
    normalized_A = (reshaped_activations_A - mean_A) / (std_A + 1e-8)  # Avoid division by zero

    # Normalize columns of B
    mean_B = reshaped_activations_B.mean(dim=0, keepdim=True)
    std_B = reshaped_activations_B.std(dim=0, keepdim=True)
    normalized_B = (reshaped_activations_B - mean_B) / (std_B + 1e-8)  # Avoid division by zero

    num_batches = (normalized_B.shape[1] + batch_size - 1) // batch_size
    max_values = []
    max_indices = []

    for batch in range(num_batches):
        start = batch * batch_size
        end = min(start + batch_size, normalized_B.shape[1])
        batch_corr_matrix = torch.matmul(normalized_A.t(), normalized_B[:, start:end]) / normalized_A.shape[0]
        max_val, max_idx = batch_corr_matrix.max(dim=0)
        max_values.append(max_val)
        max_indices.append(max_idx)

        del batch_corr_matrix
        torch.cuda.empty_cache()

    corr_inds = torch.cat(max_indices).detach().cpu().numpy()
    corr_vals = torch.cat(max_values).detach().cpu().numpy()
    return corr_inds, corr_vals

In [81]:
def filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats):
    filt_corr_ind_A = []
    filt_corr_ind_B = []
    seen = set()
    for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
        if ind_A in kept_modA_feats:
            filt_corr_ind_A.append(ind_A)
            filt_corr_ind_B.append(ind_B)
        elif ind_A not in seen:  # only keep one if it's over count X
            seen.add(ind_A)
            filt_corr_ind_A.append(ind_A)
            filt_corr_ind_B.append(ind_B)
    num_unq_pairs = len(list(set(filt_corr_ind_A)))
    print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
    print("num 1-1 feats after filt: ", num_unq_pairs )
    return filt_corr_ind_A, filt_corr_ind_B, num_unq_pairs

In [8]:
def get_new_mean_corr(modA_feats, modB_feats, corr_vals):
    new_vals = []
    seen = set()
    for ind_A, ind_B in zip(modA_feats, modB_feats):
        if ind_A not in seen:
            seen.add(ind_A)
            val = corr_vals[ind_B]
            new_vals.append(val)
    new_mean_corr = sum(new_vals) / len(new_vals)
    # print(new_mean_corr)
    return new_mean_corr

## sim fns

In [9]:
import functools
from typing import Any, Callable, Dict, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import torch


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(
    R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad"
) -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def sim_random_baseline(
    rep1: torch.Tensor, rep2: torch.Tensor, sim_func: Callable, n_permutations: int = 10
) -> Dict[str, Any]:
    torch.manual_seed(1234)
    scores = []
    for _ in range(n_permutations):
        perm = torch.randperm(rep1.size(0))

        score = sim_func(rep1[perm, :], rep2)
        score = score if isinstance(score, float) else score["score"]

        scores.append(score)

    return {"baseline_scores": np.array(scores)}


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray], Dict[str, Any]],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray) -> Dict[str, Any]:
        for preprocess_func in self.preprocess_funcs:
            R = preprocess_func(R)
            Rp = preprocess_func(Rp)
        return self.similarity_func(R, Rp)

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return (
                func.__name__
                if not isinstance(func, functools.partial)
                else func.func.__name__
            )

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )

In [10]:
from typing import List, Set, Union

import numpy as np
import numpy.typing as npt
import sklearn.neighbors
import torch

# from llmcomp.measures.utils import to_numpy_if_needed


def _jac_sim_i(idx_R: Set[int], idx_Rp: Set[int]) -> float:
    return len(idx_R.intersection(idx_Rp)) / len(idx_R.union(idx_Rp))


def jaccard_similarity(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    k: int = 10,
    inner: str = "cosine",
    n_jobs: int = 8,
) -> float:
    R, Rp = to_numpy_if_needed(R, Rp)

    indices_R = nn_array_to_setlist(top_k_neighbors(R, k, inner, n_jobs))
    indices_Rp = nn_array_to_setlist(top_k_neighbors(Rp, k, inner, n_jobs))

    return float(
        np.mean(
            [_jac_sim_i(idx_R, idx_Rp) for idx_R, idx_Rp in zip(indices_R, indices_Rp)]
        )
    )


def top_k_neighbors(
    R: npt.NDArray,
    k: int,
    inner: str,
    n_jobs: int,
) -> npt.NDArray:
    # k+1 nearest neighbors, because we pass in all the data, which means that a point
    # will be the nearest neighbor to itself. We remove this point from the results and
    # report only the k nearest neighbors distinct from the point itself.
    nns = sklearn.neighbors.NearestNeighbors(
        n_neighbors=k + 1, metric=inner, n_jobs=n_jobs
    )
    nns.fit(R)
    _, nns = nns.kneighbors(R)
    return nns[:, 1:]


def nn_array_to_setlist(nn: npt.NDArray) -> List[Set[int]]:
    return [set(idx) for idx in nn]

In [11]:
import functools
import logging
from abc import ABC
from abc import abstractmethod
from dataclasses import dataclass
from dataclasses import field
from typing import Any
from typing import Callable
from typing import get_args
from typing import List
from typing import Literal
from typing import Optional
from typing import Protocol
from typing import Tuple
from typing import Union

import numpy as np
import numpy.typing as npt
import torch
from einops import rearrange
# from loguru import logger

log = logging.getLogger(__name__)


SHAPE_TYPE = Literal["nd", "ntd", "nchw"]

ND_SHAPE, NTD_SHAPE, NCHW_SHAPE = get_args(SHAPE_TYPE)[0], get_args(SHAPE_TYPE)[1], get_args(SHAPE_TYPE)[2]


class SimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float: ...


class RSMSimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE, n_jobs: int
    ) -> float: ...


@dataclass
class BaseSimilarityMeasure(ABC):
    larger_is_more_similar: bool
    is_symmetric: bool

    is_metric: bool | None = None
    invariant_to_affine: bool | None = None
    invariant_to_invertible_linear: bool | None = None
    invariant_to_ortho: bool | None = None
    invariant_to_permutation: bool | None = None
    invariant_to_isotropic_scaling: bool | None = None
    invariant_to_translation: bool | None = None
    name: str = field(init=False)

    def __post_init__(self):
        self.name = self.__class__.__name__

    @abstractmethod
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        raise NotImplementedError


class FunctionalSimilarityMeasure(BaseSimilarityMeasure):
    @abstractmethod
    def __call__(self, output_a: torch.Tensor | npt.NDArray, output_b: torch.Tensor | npt.NDArray) -> float:
        raise NotImplementedError


@dataclass(kw_only=True)
class RepresentationalSimilarityMeasure(BaseSimilarityMeasure):
    sim_func: SimilarityFunction

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float:
        return self.sim_func(R, Rp, shape)


class RSMSimilarityMeasure(RepresentationalSimilarityMeasure):
    sim_func: RSMSimilarityFunction

    @staticmethod
    def estimate_good_number_of_jobs(R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray) -> int:
        # RSMs in are NxN (or DxD) so the number of jobs should roughly scale quadratically with increase in N (or D).
        # False! As long as sklearn-native metrics are used, they will use parallel implementations regardless of job
        # count. Each job would spawn their own threads, which leads to oversubscription of cores and thus slowdown.
        # This seems to be not fully correct (n_jobs=2 seems to actually use two cores), but using n_jobs=1 seems the
        # fastest.
        return 1

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
        n_jobs: Optional[int] = None,
    ) -> float:
        if n_jobs is None:
            n_jobs = self.estimate_good_number_of_jobs(R, Rp)
        return self.sim_func(R, Rp, shape, n_jobs=n_jobs)


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad") -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def normalize_row_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord=2, axis=1, keepdims=True)


def standardize(R: npt.NDArray) -> npt.NDArray:
    return (R - R.mean(axis=0, keepdims=True)) / R.std(axis=0)


def double_center(x: npt.NDArray) -> npt.NDArray:
    return x - x.mean(axis=0, keepdims=True) - x.mean(axis=1, keepdims=True) + x.mean()


def align_spatial_dimensions(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Aligns spatial representations by resizing them to the smallest spatial dimension.
    Subsequent aligned spatial representations are flattened, with the spatial aligned representations
    moving into the *sample* dimension.
    """
    R_re, Rp_re = resize_wh_reps(R, Rp)
    R_re = rearrange(R_re, "n c h w -> (n h w) c")
    Rp_re = rearrange(Rp_re, "n c h w -> (n h w) c")
    if R_re.shape[0] > 5000:
        logger.info(f"Got {R_re.shape[0]} samples in N after flattening. Subsampling to reduce compute.")
        subsample = R_re.shape[0] // 5000
        R_re = R_re[::subsample]
        Rp_re = Rp_re[::subsample]

    return R_re, Rp_re


def average_pool_downsample(R, resize: bool, new_size: tuple[int, int]):
    if not resize:
        return R  # do nothing
    else:
        is_numpy = isinstance(R, np.ndarray)
        R_torch = torch.from_numpy(R) if is_numpy else R
        R_torch = torch.nn.functional.adaptive_avg_pool2d(R_torch, new_size)
        return R_torch.numpy() if is_numpy else R_torch


def resize_wh_reps(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Function for resizing spatial representations that are not the same size.
    Does through fourier transform and resizing.

    Args:
        R: numpy array of shape  [batch_size, height, width, num_channels]
        RP: numpy array of shape [batch_size, height, width, num_channels]

    Returns:
        fft_acts1: numpy array of shape [batch_size, (new) height, (new) width, num_channels]
        fft_acts2: numpy array of shape [batch_size, (new) height, (new) width, num_channels]

    """
    height1, width1 = R.shape[2], R.shape[3]
    height2, width2 = Rp.shape[2], Rp.shape[3]
    if height1 != height2 or width1 != width2:
        height = min(height1, height2)
        width = min(width1, width2)
        new_size = [height, width]
        resize = True
    else:
        height = height1
        width = width1
        new_size = None
        resize = False

    # resize and preprocess with fft
    avg_ds1 = average_pool_downsample(R, resize=resize, new_size=new_size)
    avg_ds2 = average_pool_downsample(Rp, resize=resize, new_size=new_size)
    return avg_ds1, avg_ds2


def fft_resize(images, resize=False, new_size=None):
    """Function for applying DFT and resizing.

    This function takes in an array of images, applies the 2-d fourier transform
    and resizes them according to new_size, keeping the frequencies that overlap
    between the two sizes.

    Args:
              images: a numpy array with shape
                      [batch_size, height, width, num_channels]
              resize: boolean, whether or not to resize
              new_size: a tuple (size, size), with height and width the same

    Returns:
              im_fft_downsampled: a numpy array with shape
                           [batch_size, (new) height, (new) width, num_channels]
    """
    assert len(images.shape) == 4, "expecting images to be" "[batch_size, height, width, num_channels]"
    if resize:
        # FFT --> remove high frequencies --> inverse FFT
        im_complex = images.astype("complex64")
        im_fft = np.fft.fft2(im_complex, axes=(1, 2))
        im_shifted = np.fft.fftshift(im_fft, axes=(1, 2))

        center_width = im_shifted.shape[2] // 2
        center_height = im_shifted.shape[1] // 2
        half_w = new_size[0] // 2
        half_h = new_size[1] // 2
        cropped_fft = im_shifted[
            :, center_height - half_h : center_height + half_h, center_width - half_w : center_width + half_w, :
        ]
        cropped_fft_shifted_back = np.fft.ifft2(cropped_fft, axes=(1, 2))
        return cropped_fft_shifted_back.real
    else:
        return images


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray, SHAPE_TYPE], float],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray, shape: SHAPE_TYPE) -> float:
        try:
            for preprocess_func in self.preprocess_funcs:
                R = preprocess_func(R)
                Rp = preprocess_func(Rp)
            return self.similarity_func(R, Rp, shape)
        except ValueError as e:
            log.info(f"Pipeline failed: {e}")
            return np.nan

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return func.__name__ if not isinstance(func, functools.partial) else func.func.__name__

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )


def flatten(*args: Union[torch.Tensor, npt.NDArray], shape: SHAPE_TYPE) -> List[Union[torch.Tensor, npt.NDArray]]:
    if shape == "ntd":
        return list(map(flatten_nxtxd_to_ntxd, args))
    elif shape == "nd":
        return list(args)
    elif shape == "nchw":
        return list(map(flatten_nxcxhxw_to_nxchw, args))  # Flattening non-trivial for nchw
    else:
        raise ValueError("Unknown shape of representations. Must be one of 'ntd', 'nchw', 'nd'.")


def flatten_nxtxd_to_ntxd(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.flatten(R, start_dim=0, end_dim=1)
    log.debug("Shape after flattening: %s", str(R.shape))
    return R


def flatten_nxcxhxw_to_nxchw(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.reshape(R, (R.shape[0], -1))
    log.debug("Shape after flattening: %s", str(R.shape))
    return R

In [12]:
import scipy.optimize

def permutation_procrustes(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    optimal_permutation_alignment: Optional[Tuple[npt.NDArray, npt.NDArray]] = None,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    R, Rp = adjust_dimensionality(R, Rp)

    if not optimal_permutation_alignment:
        PR, PRp = scipy.optimize.linear_sum_assignment(R.T @ Rp, maximize=True)  # returns column assignments
        optimal_permutation_alignment = (PR, PRp)
    PR, PRp = optimal_permutation_alignment
    return float(np.linalg.norm(R[:, PR] - Rp[:, PRp], ord="fro"))

In [13]:
from typing import Optional
from typing import Union

import numpy as np
import numpy.typing as npt
import scipy.spatial.distance
import scipy.stats
import sklearn.metrics
import torch
# from repsim.measures.utils import flatten
# from repsim.measures.utils import RSMSimilarityMeasure
# from repsim.measures.utils import SHAPE_TYPE
# from repsim.measures.utils import to_numpy_if_needed


def representational_similarity_analysis(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    inner="correlation",
    outer="spearman",
    n_jobs: Optional[int] = None,
) -> float:
    """Representational similarity analysis

    Args:
        R (Union[torch.Tensor, npt.NDArray]): N x D representation
        Rp (Union[torch.Tensor, npt.NDArray]): N x D' representation
        inner (str, optional): inner similarity function for RSM. Must be one of
            scipy.spatial.distance.pdist identifiers . Defaults to "correlation".
        outer (str, optional): outer similarity function that compares RSMs. Defaults to
             "spearman". Must be one of "spearman", "euclidean"

    Returns:
        float: _description_
    """
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)

    if inner == "correlation":
        # n_jobs only works if metric is in PAIRWISE_DISTANCES as defined in sklearn, i.e., not for correlation.
        # But correlation = 1 - cosine dist of row-centered data, so we use the faster cosine metric and center the data.
        R = R - R.mean(axis=1, keepdims=True)
        S = scipy.spatial.distance.squareform(  # take the lower triangle of RSM
            1 - sklearn.metrics.pairwise_distances(R, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
        Rp = Rp - Rp.mean(axis=1, keepdims=True)
        Sp = scipy.spatial.distance.squareform(
            1 - sklearn.metrics.pairwise_distances(Rp, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
    elif inner == "euclidean":
        # take the lower triangle of RSM
        S = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(R, metric=inner, n_jobs=n_jobs), checks=False
        )
        Sp = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(Rp, metric=inner, n_jobs=n_jobs), checks=False
        )
    else:
        raise NotImplementedError(f"{inner=}")

    if outer == "spearman":
        return scipy.stats.spearmanr(S, Sp).statistic  # type:ignore
    elif outer == "euclidean":
        return float(np.linalg.norm(S - Sp, ord=2))
    else:
        raise ValueError(f"Unknown outer similarity function: {outer}")


class RSA(RSMSimilarityMeasure):
    def __init__(self):
        # choice of inner/outer in __call__ if fixed to default values, so these values are always the same
        super().__init__(
            sim_func=representational_similarity_analysis,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

In [14]:
##################################################################################
# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py
# Copyright 2018 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
The core code for applying Canonical Correlation Analysis to deep networks.

This module contains the core functions to apply canonical correlation analysis
to deep neural networks. The main function is get_cca_similarity, which takes in
two sets of activations, typically the neurons in two layers and their outputs
on all of the datapoints D = [d_1,...,d_m] that have been passed through.

Inputs have shape (num_neurons1, m), (num_neurons2, m). This can be directly
applied used on fully connected networks. For convolutional layers, the 3d block
of neurons can either be flattened entirely, along channels, or alternatively,
the dft_ccas (Discrete Fourier Transform) module can be used.

See:
https://arxiv.org/abs/1706.05806
https://arxiv.org/abs/1806.05759
for full details.

"""
import numpy as np
# from repsim.measures.utils import align_spatial_dimensions

num_cca_trials = 5


def positivedef_matrix_sqrt(array):
    """Stable method for computing matrix square roots, supports complex matrices.

    Args:
              array: A numpy 2d array, can be complex valued that is a positive
                     definite symmetric (or hermitian) matrix

    Returns:
              sqrtarray: The matrix square root of array
    """
    w, v = np.linalg.eigh(array)
    #  A - np.dot(v, np.dot(np.diag(w), v.T))
    wsqrt = np.sqrt(w)
    sqrtarray = np.dot(v, np.dot(np.diag(wsqrt), np.conj(v).T))
    return sqrtarray


def remove_small(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon):
    """Takes covariance between X, Y, and removes values of small magnitude.

    Args:
              sigma_xx: 2d numpy array, variance matrix for x
              sigma_xy: 2d numpy array, crossvariance matrix for x,y
              sigma_yx: 2d numpy array, crossvariance matrixy for x,y,
                        (conjugate) transpose of sigma_xy
              sigma_yy: 2d numpy array, variance matrix for y
              epsilon : cutoff value for norm below which directions are thrown
                         away

    Returns:
              sigma_xx_crop: 2d array with low x norm directions removed
              sigma_xy_crop: 2d array with low x and y norm directions removed
              sigma_yx_crop: 2d array with low x and y norm directiosn removed
              sigma_yy_crop: 2d array with low y norm directions removed
              x_idxs: indexes of sigma_xx that were removed
              y_idxs: indexes of sigma_yy that were removed
    """

    x_diag = np.abs(np.diagonal(sigma_xx))
    y_diag = np.abs(np.diagonal(sigma_yy))
    x_idxs = x_diag >= epsilon
    y_idxs = y_diag >= epsilon

    sigma_xx_crop = sigma_xx[x_idxs][:, x_idxs]
    sigma_xy_crop = sigma_xy[x_idxs][:, y_idxs]
    sigma_yx_crop = sigma_yx[y_idxs][:, x_idxs]
    sigma_yy_crop = sigma_yy[y_idxs][:, y_idxs]

    return (sigma_xx_crop, sigma_xy_crop, sigma_yx_crop, sigma_yy_crop, x_idxs, y_idxs)


def compute_ccas(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon, verbose=True):
    """Main cca computation function, takes in variances and crossvariances.

    This function takes in the covariances and cross covariances of X, Y,
    preprocesses them (removing small magnitudes) and outputs the raw results of
    the cca computation, including cca directions in a rotated space, and the
    cca correlation coefficient values.

    Args:
              sigma_xx: 2d numpy array, (num_neurons_x, num_neurons_x)
                        variance matrix for x
              sigma_xy: 2d numpy array, (num_neurons_x, num_neurons_y)
                        crossvariance matrix for x,y
              sigma_yx: 2d numpy array, (num_neurons_y, num_neurons_x)
                        crossvariance matrix for x,y (conj) transpose of sigma_xy
              sigma_yy: 2d numpy array, (num_neurons_y, num_neurons_y)
                        variance matrix for y
              epsilon:  small float to help with stabilizing computations
              verbose:  boolean on whether to print intermediate outputs

    Returns:
              [ux, sx, vx]: [numpy 2d array, numpy 1d array, numpy 2d array]
                            ux and vx are (conj) transposes of each other, being
                            the canonical directions in the X subspace.
                            sx is the set of canonical correlation coefficients-
                            how well corresponding directions in vx, Vy correlate
                            with each other.
              [uy, sy, vy]: Same as above, but for Y space
              invsqrt_xx:   Inverse square root of sigma_xx to transform canonical
                            directions back to original space
              invsqrt_yy:   Same as above but for sigma_yy
              x_idxs:       The indexes of the input sigma_xx that were pruned
                            by remove_small
              y_idxs:       Same as above but for sigma_yy
    """

    (sigma_xx, sigma_xy, sigma_yx, sigma_yy, x_idxs, y_idxs) = remove_small(
        sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon
    )

    numx = sigma_xx.shape[0]
    numy = sigma_yy.shape[0]

    if numx == 0 or numy == 0:
        return (
            [0, 0, 0],
            [0, 0, 0],
            np.zeros_like(sigma_xx),
            np.zeros_like(sigma_yy),
            x_idxs,
            y_idxs,
        )

    if verbose:
        print("adding eps to diagonal and taking inverse")
    sigma_xx += epsilon * np.eye(numx)
    sigma_yy += epsilon * np.eye(numy)
    inv_xx = np.linalg.pinv(sigma_xx)
    inv_yy = np.linalg.pinv(sigma_yy)

    if verbose:
        print("taking square root")
    invsqrt_xx = positivedef_matrix_sqrt(inv_xx)
    invsqrt_yy = positivedef_matrix_sqrt(inv_yy)

    if verbose:
        print("dot products...")
    arr = np.dot(invsqrt_xx, np.dot(sigma_xy, invsqrt_yy))

    if verbose:
        print("trying to take final svd")
    u, s, v = np.linalg.svd(arr)

    if verbose:
        print("computed everything!")

    return [u, np.abs(s), v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs


def sum_threshold(array, threshold):
    """Computes threshold index of decreasing nonnegative array by summing.

    This function takes in a decreasing array nonnegative floats, and a
    threshold between 0 and 1. It returns the index i at which the sum of the
    array up to i is threshold*total mass of the array.

    Args:
              array: a 1d numpy array of decreasing, nonnegative floats
              threshold: a number between 0 and 1

    Returns:
              i: index at which np.sum(array[:i]) >= threshold
    """
    assert (threshold >= 0) and (threshold <= 1), "print incorrect threshold"

    for i in range(len(array)):
        if np.sum(array[:i]) / np.sum(array) >= threshold:
            return i


def create_zero_dict(compute_dirns, dimension):
    """Outputs a zero dict when neuron activation norms too small.

    This function creates a return_dict with appropriately shaped zero entries
    when all neuron activations are very small.

    Args:
              compute_dirns: boolean, whether to have zero vectors for directions
              dimension: int, defines shape of directions

    Returns:
              return_dict: a dict of appropriately shaped zero entries
    """
    return_dict = {}
    return_dict["mean"] = (np.asarray(0), np.asarray(0))
    return_dict["sum"] = (np.asarray(0), np.asarray(0))
    return_dict["cca_coef1"] = np.asarray(0)
    return_dict["cca_coef2"] = np.asarray(0)
    return_dict["idx1"] = 0
    return_dict["idx2"] = 0

    if compute_dirns:
        return_dict["cca_dirns1"] = np.zeros((1, dimension))
        return_dict["cca_dirns2"] = np.zeros((1, dimension))

    return return_dict


def get_cca_similarity(
    acts1,
    acts2,
    epsilon=0.0,
    threshold=0.98,
    compute_coefs=True,
    compute_dirns=False,
    verbose=True,
):
    """The main function for computing cca similarities.

    This function computes the cca similarity between two sets of activations,
    returning a dict with the cca coefficients, a few statistics of the cca
    coefficients, and (optionally) the actual directions.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              epsilon: small float to help stabilize computations

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.


              compute_coefs: boolean value determining whether coefficients
                             over neurons are computed. Needed for computing
                             directions

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

              verbose: Boolean, whether intermediate outputs are printed

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    # assert dimensionality equal
    assert acts1.shape[1] == acts2.shape[1], "dimensions don't match"
    # check that acts1, acts2 are transposition
    assert acts1.shape[0] < acts1.shape[1], "input must be number of neurons" "by datapoints"
    return_dict = {}

    # compute covariance with numpy function for extra stability
    numx = acts1.shape[0]
    numy = acts2.shape[0]

    covariance = np.cov(acts1, acts2)
    sigmaxx = covariance[:numx, :numx]
    sigmaxy = covariance[:numx, numx:]
    sigmayx = covariance[numx:, :numx]
    sigmayy = covariance[numx:, numx:]

    # rescale covariance to make cca computation more stable
    xmax = np.max(np.abs(sigmaxx))
    ymax = np.max(np.abs(sigmayy))
    sigmaxx /= xmax
    sigmayy /= ymax
    sigmaxy /= np.sqrt(xmax * ymax)
    sigmayx /= np.sqrt(xmax * ymax)

    ([u, s, v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs) = compute_ccas(
        sigmaxx, sigmaxy, sigmayx, sigmayy, epsilon=epsilon, verbose=verbose
    )

    # if x_idxs or y_idxs is all false, return_dict has zero entries
    if (not np.any(x_idxs)) or (not np.any(y_idxs)):
        return create_zero_dict(compute_dirns, acts1.shape[1])

    if compute_coefs:
        # also compute full coefficients over all neurons
        x_mask = np.dot(x_idxs.reshape((-1, 1)), x_idxs.reshape((1, -1)))
        y_mask = np.dot(y_idxs.reshape((-1, 1)), y_idxs.reshape((1, -1)))

        return_dict["coef_x"] = u.T
        return_dict["invsqrt_xx"] = invsqrt_xx
        return_dict["full_coef_x"] = np.zeros((numx, numx))
        np.place(return_dict["full_coef_x"], x_mask, return_dict["coef_x"])
        return_dict["full_invsqrt_xx"] = np.zeros((numx, numx))
        np.place(return_dict["full_invsqrt_xx"], x_mask, return_dict["invsqrt_xx"])

        return_dict["coef_y"] = v
        return_dict["invsqrt_yy"] = invsqrt_yy
        return_dict["full_coef_y"] = np.zeros((numy, numy))
        np.place(return_dict["full_coef_y"], y_mask, return_dict["coef_y"])
        return_dict["full_invsqrt_yy"] = np.zeros((numy, numy))
        np.place(return_dict["full_invsqrt_yy"], y_mask, return_dict["invsqrt_yy"])

        # compute means
        neuron_means1 = np.mean(acts1, axis=1, keepdims=True)
        neuron_means2 = np.mean(acts2, axis=1, keepdims=True)
        return_dict["neuron_means1"] = neuron_means1
        return_dict["neuron_means2"] = neuron_means2

    if compute_dirns:
        # orthonormal directions that are CCA directions
        cca_dirns1 = (
            np.dot(
                np.dot(return_dict["full_coef_x"], return_dict["full_invsqrt_xx"]),
                (acts1 - neuron_means1),
            )
            + neuron_means1
        )
        cca_dirns2 = (
            np.dot(
                np.dot(return_dict["full_coef_y"], return_dict["full_invsqrt_yy"]),
                (acts2 - neuron_means2),
            )
            + neuron_means2
        )

    # get rid of trailing zeros in the cca coefficients
    idx1 = sum_threshold(s, threshold)
    idx2 = sum_threshold(s, threshold)

    return_dict["cca_coef1"] = s
    return_dict["cca_coef2"] = s
    return_dict["x_idxs"] = x_idxs
    return_dict["y_idxs"] = y_idxs
    # summary statistics
    return_dict["mean"] = (np.mean(s[:idx1]), np.mean(s[:idx2]))
    return_dict["sum"] = (np.sum(s), np.sum(s))

    if compute_dirns:
        return_dict["cca_dirns1"] = cca_dirns1
        return_dict["cca_dirns2"] = cca_dirns2

    return return_dict


def robust_cca_similarity(acts1, acts2, threshold=0.98, epsilon=1e-6, compute_dirns=True):
    """Calls get_cca_similarity multiple times while adding noise.

    This function is very similar to get_cca_similarity, and can be used if
    get_cca_similarity doesn't converge for some pair of inputs. This function
    adds some noise to the activations to help convergence.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.

              epsilon: small float to help stabilize computations

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    for trial in range(num_cca_trials):
        try:
            return_dict = get_cca_similarity(acts1, acts2, threshold, compute_dirns)
        except np.linalg.LinAlgError:
            acts1 = acts1 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            acts2 = acts2 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            if trial + 1 == num_cca_trials:
                raise

    return return_dict
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py


def top_k_pca_comps(singular_values, threshold=0.99):
    total_variance = np.sum(singular_values**2)
    explained_variance = (singular_values**2) / total_variance
    cumulative_variance = np.cumsum(explained_variance)
    return np.argmax(cumulative_variance >= threshold * total_variance) + 1


def _svcca_original(acts1, acts2):
    # Copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    # Modification: get_cca_similarity is in the same file.
    # Modification: top-k PCA component selection s.t. explained variance > 0.99 total variance
    # Mean subtract activations
    cacts1 = acts1 - np.mean(acts1, axis=1, keepdims=True)
    cacts2 = acts2 - np.mean(acts2, axis=1, keepdims=True)

    # Perform SVD
    U1, s1, V1 = np.linalg.svd(cacts1, full_matrices=False)
    U2, s2, V2 = np.linalg.svd(cacts2, full_matrices=False)

    # top-k PCA components only
    k1 = top_k_pca_comps(s1)
    k2 = top_k_pca_comps(s2)

    svacts1 = np.dot(s1[:k1] * np.eye(k1), V1[:k1])
    # can also compute as svacts1 = np.dot(U1.T[:20], cacts1)
    svacts2 = np.dot(s2[:k2] * np.eye(k2), V2[:k2])
    # can also compute as svacts1 = np.dot(U2.T[:20], cacts2)

    svcca_results = get_cca_similarity(svacts1, svacts2, epsilon=1e-10, verbose=False)
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    return np.mean(svcca_results["cca_coef1"])


# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py
# Modification: get_cca_similarity is in the same file.
def compute_pwcca(acts1, acts2, epsilon=0.0):
    """Computes projection weighting for weighting CCA coefficients

    Args:
         acts1: 2d numpy array, shaped (neurons, num_datapoints)
         acts2: 2d numpy array, shaped (neurons, num_datapoints)

    Returns:
         Original cca coefficient mean and weighted mean

    """
    sresults = get_cca_similarity(
        acts1,
        acts2,
        epsilon=epsilon,
        compute_dirns=False,
        compute_coefs=True,
        verbose=False,
    )
    if np.sum(sresults["x_idxs"]) <= np.sum(sresults["y_idxs"]):
        dirns = (
            np.dot(
                sresults["coef_x"],
                (acts1[sresults["x_idxs"]] - sresults["neuron_means1"][sresults["x_idxs"]]),
            )
            + sresults["neuron_means1"][sresults["x_idxs"]]
        )
        coefs = sresults["cca_coef1"]
        acts = acts1
        idxs = sresults["x_idxs"]
    else:
        dirns = (
            np.dot(
                sresults["coef_y"],
                (acts1[sresults["y_idxs"]] - sresults["neuron_means2"][sresults["y_idxs"]]),
            )
            + sresults["neuron_means2"][sresults["y_idxs"]]
        )
        coefs = sresults["cca_coef2"]
        acts = acts2
        idxs = sresults["y_idxs"]
    P, _ = np.linalg.qr(dirns.T)
    weights = np.sum(np.abs(np.dot(P.T, acts[idxs].T)), axis=1)
    weights = weights / np.sum(weights)

    return np.sum(weights * coefs), weights, coefs
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py


##################################################################################

from typing import Union  # noqa:e402

import numpy.typing as npt  # noqa:e402
import torch  # noqa:e402

# from repsim.measures.utils import (
#     SHAPE_TYPE,
#     flatten,
#     resize_wh_reps,
#     to_numpy_if_needed,
#     RepresentationalSimilarityMeasure,
# )  # noqa:e402


def svcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return _svcca_original(R.T, Rp.T)


def pwcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return compute_pwcca(R.T, Rp.T)[0]


class SVCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=svcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=True,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)


class PWCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=pwcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=False,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=False,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)

## get rand

In [15]:
def score_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    i = 0
    # for i in range(num_runs):
    while i < num_runs:
        try:
            rand_modA_feats = np.random.choice(range(weight_matrix_np.shape[0]), size=num_feats, replace=False).tolist()
            rand_modB_feats = np.random.choice(range(weight_matrix_2.shape[0]), size=num_feats, replace=False).tolist()

            if shapereq_bool:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
            else:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
            all_rand_scores.append(score)
            i += 1
        except:
            continue
    return sum(all_rand_scores) / len(all_rand_scores)

In [16]:
def score_rand_corr(num_runs, weight_matrix_np, weight_matrix_2, num_feats, highest_correlations_indices_AB, sim_fn, shapereq_bool):
    all_rand_scores = []
    i = 0
    # for i in range(num_runs):
    while i < num_runs:
        try:
            rand_modB_feats = np.random.choice(range(weight_matrix_2.shape[0]), size=num_feats, replace=False).tolist()
            rand_modA_feats = [highest_correlations_indices_AB[index] for index in rand_modB_feats]

            if shapereq_bool:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
            else:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
            all_rand_scores.append(score)
            i += 1
        except:
            continue
    # print(sum(all_rand_scores) / len(all_rand_scores))
    # plt.hist(all_rand_scores)
    # plt.show()
    return all_rand_scores

In [17]:
import random
def shuffle_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    for i in range(num_runs):
        row_idxs = list(range(num_feats))
        random.shuffle(row_idxs)
        if shapereq_bool:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs], "nd")
        else:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs])
        all_rand_scores.append(score)
    # return sum(all_rand_scores) / len(all_rand_scores)
    return all_rand_scores

## plot fns

In [18]:
def plot_svcca_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['svcca_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('SVCCA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [19]:
def plot_rsa_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['rsa_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['rsa_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('RSA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [20]:
def plot_meanCorr_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr'] for i in range(0, 12)]
    # unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    # rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    # label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [21]:
def plot_meanCorr_filt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [22]:
def plot_numFeats_afterFilt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['num_feat_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Num Feats Kept')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [23]:
# def plot_js_byLayer(layer_to_dictscores):
#     for key, sub_dict in layer_to_dictscores.items():
#         for sub_key, value in sub_dict.items():
#             sub_dict[sub_key] = round(value, 4)

#     layers = [f'L{i}' for i in range(0, 12)]
#     paired_values = [layer_to_dictscores[i]['1-1 jaccard_paired'] for i in range(0, 12)]
#     unpaired_values = [layer_to_dictscores[i]['1-1 jaccard_unpaired'] for i in range(0, 12)]

#     # Plotting configuration
#     x = np.arange(len(layers))  # label locations
#     width = 0.35  # width of the bars

#     # Increase figure size
#     fig, ax = plt.subplots(figsize=(12, 7))  # Slightly increased figure size
#     rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
#     rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

#     # Adding labels, title and custom x-axis tick labels
#     ax.set_ylabel('Jaccard NN')
#     ax.set_title('SAEs comparison by Pythia 70m MLP0 vs 160m MLP Layers')
#     ax.set_xticks(x)
#     ax.set_xticklabels(layers)
#     ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
#     ax.legend()

#     # Label bars with increased font size and different positioning for paired and unpaired
#     def label_bars(rects, is_paired):
#         for rect in rects:
#             height = rect.get_height()
#             label_height = height + 0.05 if is_paired else height + 0.01
#             ax.text(rect.get_x() + rect.get_width()/2., label_height,
#                     f'{height:.3f}',
#                     ha='center', va='bottom', fontsize=9)  # Increased font size to 12

#     label_bars(rects1, True)   # Paired bars
#     label_bars(rects2, False)  # Unpaired bars

#     # Adjust layout to prevent cutting off labels
#     plt.tight_layout()

#     # Increase y-axis limit to accommodate higher labels
#     ax.set_ylim(0, 1.1)  # Increased from 1 to 1.1

#     plt.show()

## interpret fns

In [24]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
): # -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [25]:
def store_top_toks(top_acts_indices, top_acts_values, batch_tokens):
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        new_str_token = tokenizer.decode(batch_tokens[batch_idx, seq_idx]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
        feat_samps.append(new_str_token)
    return feat_samps

In [26]:
def find_indices_with_keyword(fList, keyword):
    """
    Find all indices of fList which contain the keyword in the string at those indices.

    Args:
    fList (list of str): List of strings to search within.
    keyword (str): Keyword to search for within the strings of fList.

    Returns:
    list of int: List of indices where the keyword is found within the strings of fList.
    """
    index_list = []
    for index, split_list in enumerate(fList):
        no_space_list = [i.replace(' ', '').lower() for i in split_list]
        if keyword in no_space_list:
            index_list.append(index)
    return index_list

## get concept space features

In [44]:
# def get_mixed_feats(keywords):
def get_mixed_feats(fList_model_B, corr_inds, keywords):
    mixed_modA_feats = []
    mixed_modB_feats = []
    for kw in keywords:
        modB_feats = find_indices_with_keyword(fList_model_B, kw)
        modA_feats = [corr_inds[index] for index in modB_feats]
        mixed_modA_feats.extend(modA_feats)
        mixed_modB_feats.extend(modB_feats)

    print("Unique modA feats: ", len(list(set(mixed_modA_feats))) )
    print("Unique modB feats: ",  len(list(set(mixed_modB_feats))) )
    return mixed_modA_feats, mixed_modB_feats

## get actv fns

In [29]:
# def get_weights_and_acts(name, cfg_dict, layer_id, outputs):
def get_weights_and_acts(name, layer_id, outputs):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    hookpoint = "layers." + str(layer_id)

    sae = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

    weight_matrix = sae.W_dec.cpu().detach().numpy()

    with torch.inference_mode():
        # reshaped_activations_A = sae.pre_acts(outputs.to("cuda"))
        reshaped_activations = sae.pre_acts(outputs.hidden_states[layer_id].to("cuda"))

    first_dim_reshaped = reshaped_activations.shape[0] * reshaped_activations.shape[1]
    reshaped_activations = reshaped_activations.reshape(first_dim_reshaped, reshaped_activations.shape[-1]).cpu()

    # return weight_matrix_np, reshaped_activations_A, orig
    return weight_matrix, reshaped_activations

In [30]:
def count_zero_columns(tensor):
    # Check if all elements in each column are zero
    zero_columns = np.all(tensor == 0, axis=0)
    # Count True values in the zero_columns array
    zero_cols_indices = np.where(zero_columns)[0]
    return np.sum(zero_columns), zero_cols_indices

## run expm fns

In [31]:
def run_expm(layer_id, outputs, outputs_2):
    layer_to_dictscores = {}

    name = "EleutherAI/sae-pythia-70m-32k"
    cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 512}
    weight_matrix_np, reshaped_activations_A = get_weights_and_acts(name, cfg_dict, layer_id, outputs)
    # zero_cols_count, zero_cols_indices = count_zero_columns(reshaped_activations_A.cpu().numpy())
    # print("Number of zero columns:", zero_cols_count) #, zero_cols_indices

    name = "EleutherAI/sae-pythia-160m-32k"
    for layerID_2 in range(0, 12): # 0, 12
        dictscores = {}

        # redef
        cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 768}
        weight_matrix_2, reshaped_activations_B = get_weights_and_acts(name, cfg_dict, layerID_2, outputs_2)


        highest_correlations_indices_AB, highest_correlations_values_AB = batched_correlation(reshaped_activations_A, reshaped_activations_B)

        num_unq_pairs = len(list(set(highest_correlations_indices_AB)))
        print("% unique: ", num_unq_pairs / len(highest_correlations_indices_AB))

        dictscores["mean_actv_corr"] = sum(highest_correlations_values_AB) / len(highest_correlations_values_AB)

        ###########
        # filter

        sorted_feat_counts = Counter(highest_correlations_indices_AB).most_common()
        # kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 100]
        kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 1]

        filt_corr_ind_A = []
        filt_corr_ind_B = []
        seen = set()
        for ind_B, ind_A in enumerate(highest_correlations_indices_AB):
            if ind_A in kept_modA_feats:
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
            elif ind_A not in seen:  # only keep one if it's over count X
                seen.add(ind_A)
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
        # num_unq_pairs = len(list(set(filt_corr_ind_A)))
        # print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
        # print("num feats after filt: ", len(filt_corr_ind_A))

        new_highest_correlations_indices_A = []
        new_highest_correlations_indices_B = []
        new_highest_correlations_values = []

        for ind_A, ind_B in zip(filt_corr_ind_A, filt_corr_ind_B):
            val = highest_correlations_values_AB[ind_B]
            if val > 0:
                new_highest_correlations_indices_A.append(ind_A)
                new_highest_correlations_indices_B.append(ind_B)
                new_highest_correlations_values.append(val)

        # print("% unique after rmv 0s: ", num_unq_pairs / len(new_highest_correlations_indices_A))
        # print("num feats after rmv 0s: ", len(new_highest_correlations_indices_A))
        dictscores["num_feat_kept"] = len(new_highest_correlations_indices_A)

        dictscores["mean_actv_corr_filt"] = sum(new_highest_correlations_values) / len(new_highest_correlations_values)

        ###########
        # sim tests

        # # num_feats = len(filt_corr_ind_A)
        num_feats = len(new_highest_correlations_indices_A)

        # # dictscores["svcca_paired"] = svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")
        # dictscores["svcca_paired"] = svcca(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")

        # dictscores["svcca_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
        #                                           svcca, shapereq_bool=True)

        dictscores["rsa_paired"] = representational_similarity_analysis(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")
        dictscores["rsa_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
                                                  representational_similarity_analysis, shapereq_bool=True)

        print("Layer: " + str(layerID_2))
        for key, value in dictscores.items():
            print(key + ": " + str(value))
        print("\n")

        layer_to_dictscores[layerID_2] = dictscores
    return layer_to_dictscores

# load data

In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
from datasets import load_dataset
# dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)
dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True)

openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [34]:
batch_size = 100
maxseqlen = 300

def get_next_batch(dataset_iter, batch_size=100):
    batch = []
    for _ in range(batch_size):
        try:
            sample = next(dataset_iter)
            batch.append(sample['text'])
        except StopIteration:
            break
    return batch

dataset_iter = iter(dataset)
batch = get_next_batch(dataset_iter, batch_size)

# Tokenize the batch
inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=maxseqlen)

# load models

In [35]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
model_2 = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

## get LLM actvs

In [36]:
with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True)
    outputs_2 = model_2(**inputs, output_hidden_states=True)

# concept keywords

In [37]:
keywords = {}

keywords['numeric'] = [
    "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
    "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty",
    "hundred", "thousand", "million", "billion", "trillion",
    "integer", "fraction", "decimal", "percentage", "ratio",
    "numeral", "digit", "prime", "even", "odd",
    "sum", "difference", "product", "factor", "multiple",
    "total", "count", "measure", "dozen", "score", "unit"]

keywords['people'] = [
    "man", "girl", "boy", "kid", "dad", "mom", "son", "sis", "bro",
    "pal", "mate", "boss", "chief", "cop", "guide", "priest", "king",
    "queen", "duke", "lord", "friend", "judge", "clerk", "coach", "team",
    "crew", "staff", "nurse", "doc", "vet", "cook", "maid", "clown",
    "star", "clan", "host", "guest", "peer", "guard", "boss", "spy",
    "fool", "punk", "nerd", "jock", "chief", "folk", "crowd"]

keywords['nature'] = [
    "tree", "grass", "bush", "plant", "stone", "rock", "cliff", "hill",
    "dirt", "sand", "mud", "wind", "storm", "rain", "cloud", "sun",
    "moon", "star", "leaf", "branch", "twig", "root", "bark", "seed",
    "wave", "tide", "lake", "pond", "creek", "sea", "wood", "field",
    "shore", "snow", "ice", "flame", "fire", "fog", "dew", "hail",
    "sky", "earth", "glade", "cave", "peak", "ridge", "dust", "air",
    "mist", "heat"]

keywords['science'] = [
    "cell", "gene", "nerve", "brain", "blood", "bone", "heart", "lung",
    "star", "space", "light", "mass", "force", "wave", "speed", "sound",
    "time", "power", "heat", "cold", "charge", "spark", "flame", "bond",
    "quark", "atom", "ion", "gas", "wind", "ice", "plant", "rock",
    "probe", "test", "fact", "proof", "code", "law", "rule", "graph",
    "scale", "scope", "lens", "ray", "line", "chart", "flux", "phase",
    "shock", "pulse"]

keywords['animals'] = [
    "dog", "cat", "rat", "bat", "pig", "cow", "fox", "wolf", "ram", "eel",
    "ant", "bee", "bug", "cub", "kit", "fawn", "calf", "colt", "foal",
    "hen", "duck", "goat", "bird", "crow", "fish", "frog", "deer", "worm",
    "moth", "gnat", "clam", "crab", "shrimp", "whale", "shark", "squid",
    "pup", "joey", "owl", "hare", "seal", "mule", "toad", "swan", "sow",
    "bull", "stag", "buck", "boar", "kite"
]

# run expm fn

In [88]:
# def run_expm(layer_id, outputs, outputs_2):
def run_semantic_subspace_expms(fList_model_B, outputs):
    metric_dict = {'SVCCA': svcca, 'RSA': representational_similarity_analysis}
    # map_type_list = ['manyTo1', '1To1']
    map_type_list = ['1To1']
    layer_to_dictscores = {}

    name = "EleutherAI/sae-pythia-70m-32k"
    for layerID_2 in range(0, 6):
        print("Layer: " + str(layerID_2))
        dictscores = {}

        weight_matrix_np, reshaped_activations_A = get_weights_and_acts(name, layerID_2, outputs)

        corr_inds, corr_vals = batched_correlation(reshaped_activations_A, reshaped_activations_B)
        sorted_feat_counts = Counter(corr_inds).most_common()
        kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count == 1]

        all_scores_dict = {}
        for concept_name in keywords.keys():
            print(concept_name)
            scores_dict = {}

            mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B, corr_inds, keywords[concept_name])
            filt_corr_ind_A, filt_corr_ind_B, num_unq_pairs = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

            scores_dict[f'num_unq_pairs'] = num_unq_pairs

            for map_type in map_type_list:
                print(map_type)
                if map_type == '1To1':
                    X_subset = weight_matrix_np[filt_corr_ind_A]
                    Y_subset = weight_matrix_2[filt_corr_ind_B]
                    num_feats = len(list(set(filt_corr_ind_A)))
                else:
                    X_subset = weight_matrix_np[mixed_modA_feats]
                    Y_subset = weight_matrix_2[mixed_modB_feats]
                    num_feats = len(list(set(mixed_modA_feats)))

                scores_dict[f'mean_corr_{map_type}'] = get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

                for metric_name, metric_fn in metric_dict.items():
                    paired_score = metric_fn(X_subset, Y_subset, "nd")
                    scores_dict[f'paired_{metric_name}_{map_type}'] = paired_score

                    rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                                    num_feats, corr_inds, metric_fn, True)
                    scores_dict[f'rand_corr_mean_{metric_name}_{map_type}'] = sum(rand_corr_scores) / len(rand_corr_scores)
                    scores_dict[f'rand_corr_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_corr_scores) >= paired_score)

                    rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                                    metric_fn, shapereq_bool=True)
                    scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = sum(rand_shuff_scores) / len(rand_shuff_scores)
                    scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_shuff_scores) >= paired_score)

            all_scores_dict[concept_name] = scores_dict
            print('\n')

        # for key, value in all_scores_dict.items():
        #     print(key + ": " + str(value))
        # print("\n")

        layer_to_dictscores[layerID_2] = all_scores_dict
    return layer_to_dictscores

# loop for any layer pair: L10

In [39]:
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
layer_id_2 = 10

hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

layers.10/cfg.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [40]:
# store feature : lst of top strs
fList_model_B = []
samp_m = 5

for feature_idx in range(feature_acts_B.shape[-1]):
    if feature_idx % 5000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  5000
Feature:  10000
Feature:  15000
Feature:  20000
Feature:  25000
Feature:  30000


In [41]:
import pickle
from google.colab import files
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(fList_model_B, f)
files.download(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
# with open('fList_model_B.pkl', 'rb') as f:
#     fList_model_B = pickle.load(f)

In [45]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B, outputs)

Layer: 0


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  58
Unique modB feats:  717
% unique:  1.0
num 1-1 feats after filt:  58
1To1


people
Unique modA feats:  48
Unique modB feats:  233
% unique:  1.0
num 1-1 feats after filt:  48
1To1


nature
Unique modA feats:  52
Unique modB feats:  304
% unique:  1.0
num 1-1 feats after filt:  52
1To1


science
Unique modA feats:  58
Unique modB feats:  1810
% unique:  1.0
num 1-1 feats after filt:  58
1To1


animals
Unique modA feats:  34
Unique modB feats:  79
% unique:  1.0
num 1-1 feats after filt:  34
1To1


Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

...

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

numeric
Unique modA feats:  457
Unique modB feats:  717
% unique:  0.9682203389830508
num 1-1 feats after filt:  457
1To1


people
Unique modA feats:  182
Unique modB feats:  233
% unique:  0.9837837837837838
num 1-1 feats after filt:  182
1To1


nature
Unique modA feats:  236
Unique modB feats:  304
% unique:  0.9957805907172996
num 1-1 feats after filt:  236
1To1


science
Unique modA feats:  757
Unique modB feats:  1810
% unique:  0.9869621903520208
num 1-1 feats after filt:  757
1To1


animals
Unique modA feats:  70
Unique modB feats:  79
% unique:  1.0
num 1-1 feats after filt:  70
1To1


Layer: 5


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

numeric
Unique modA feats:  489
Unique modB feats:  717
% unique:  0.9741035856573705
num 1-1 feats after filt:  489
1To1


people
Unique modA feats:  196
Unique modB feats:  233
% unique:  0.98
num 1-1 feats after filt:  196
1To1


nature
Unique modA feats:  250
Unique modB feats:  304
% unique:  0.9960159362549801
num 1-1 feats after filt:  250
1To1


science
Unique modA feats:  741
Unique modB feats:  1810
% unique:  0.9959677419354839
num 1-1 feats after filt:  741
1To1


animals
Unique modA feats:  75
Unique modB feats:  79
% unique:  1.0
num 1-1 feats after filt:  75
1To1




In [46]:
with open(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L0

In [52]:
all_scores_dict = layer_to_dictscores[0]

In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

In [53]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.054796
nature,0.036543
numeric,0.051398
people,0.039010
science,0.048151


In [54]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.12,0.13,0.48,0.11,0.40
people,0.02,0.13,0.89,0.13,0.90
nature,0.02,0.15,0.96,0.12,0.90
science,0.01,0.15,0.93,0.12,0.94
animals,0.05,0.15,0.78,0.15,0.82


In [55]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.01,0.02,0.47,-0.00,0.24
people,0.01,0.01,0.58,-0.00,0.34
nature,-0.00,0.01,0.69,-0.00,0.48
science,-0.03,0.02,0.96,0.00,0.93
animals,-0.03,0.01,0.79,-0.01,0.68


## analyze data: L1

In [56]:
all_scores_dict = layer_to_dictscores[1]

In [57]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.443858
nature,0.383602
numeric,0.298403
people,0.353961
science,0.367486


In [58]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.10,0.15,0.88,0.04,0.07
people,0.29,0.15,0.07,0.06,0.00
nature,0.34,0.13,0.00,0.06,0.00
science,0.16,0.14,0.37,0.04,0.00
animals,0.25,0.14,0.17,0.10,0.04


In [59]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.04,1.00,0.0,0.0
people,0.06,0.04,0.00,-0.0,0.0
nature,0.07,0.04,0.01,-0.0,0.0
science,0.03,0.04,0.69,0.0,0.0
animals,0.08,0.03,0.02,-0.0,0.0


## analyze data: L2

In [60]:
all_scores_dict = layer_to_dictscores[2]

In [61]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.446678
nature,0.402963
numeric,0.327972
people,0.366059
science,0.414025


In [62]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.15,0.05,0.01,0.04,0.00
people,0.00,0.06,0.99,0.06,0.98
nature,0.33,0.06,0.00,0.05,0.00
science,0.11,0.04,0.05,0.03,0.03
animals,0.10,0.10,0.38,0.09,0.37


In [63]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.03,0.02,0.05,-0.0,0.00
people,0.05,0.02,0.00,0.0,0.00
nature,0.05,0.02,0.00,-0.0,0.00
science,0.05,0.02,0.00,0.0,0.00
animals,0.05,0.02,0.08,0.0,0.04


## analyze data: L3

In [64]:
all_scores_dict = layer_to_dictscores[3]

In [65]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.442887
nature,0.374600
numeric,0.316841
people,0.389472
science,0.387279


In [66]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.08,0.07,0.40,0.04,0.17
people,0.48,0.08,0.00,0.06,0.00
nature,0.45,0.07,0.00,0.05,0.00
science,0.11,0.06,0.14,0.04,0.01
animals,0.06,0.11,0.63,0.12,0.69


In [67]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.05,0.03,0.01,0.0,0.0
people,0.10,0.04,0.00,-0.0,0.0
nature,0.08,0.03,0.00,-0.0,0.0
science,0.07,0.03,0.00,0.0,0.0
animals,0.09,0.04,0.01,-0.0,0.0


## analyze data: L4

In [68]:
all_scores_dict = layer_to_dictscores[4]

In [69]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.292956
nature,0.277071
numeric,0.327714
people,0.303845
science,0.473705


In [70]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.09,0.06,0.19,0.04,0.08
people,0.13,0.07,0.15,0.06,0.07
nature,0.21,0.06,0.00,0.05,0.00
science,0.25,0.06,0.00,0.03,0.00
animals,0.11,0.09,0.36,0.09,0.34


In [71]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.03,0.01,0.00,0.0,0.00
people,0.05,0.01,0.00,0.0,0.00
nature,0.05,0.02,0.00,-0.0,0.00
science,0.03,0.01,0.00,0.0,0.00
animals,0.03,0.01,0.18,-0.0,0.06


## analyze data: L5

In [72]:
all_scores_dict = layer_to_dictscores[4]

In [73]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.292956
nature,0.277071
numeric,0.327714
people,0.303845
science,0.473705


In [74]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.09,0.06,0.19,0.04,0.08
people,0.13,0.07,0.15,0.06,0.07
nature,0.21,0.06,0.00,0.05,0.00
science,0.25,0.06,0.00,0.03,0.00
animals,0.11,0.09,0.36,0.09,0.34


In [75]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.03,0.01,0.00,0.0,0.00
people,0.05,0.01,0.00,0.0,0.00
nature,0.05,0.02,0.00,-0.0,0.00
science,0.03,0.01,0.00,0.0,0.00
animals,0.03,0.01,0.18,-0.0,0.06


# loop for any layer pair: L2

In [76]:
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
layer_id_2 = 2

hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [77]:
# store feature : lst of top strs
fList_model_B = []
samp_m = 5

for feature_idx in range(feature_acts_B.shape[-1]):
    if feature_idx % 5000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  5000
Feature:  10000
Feature:  15000
Feature:  20000
Feature:  25000
Feature:  30000


In [78]:
import pickle
from google.colab import files
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(fList_model_B, f)
files.download(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
# with open('fList_model_B.pkl', 'rb') as f:
#     fList_model_B = pickle.load(f)

In [89]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B, outputs)

Layer: 0


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  48
Unique modB feats:  417
% unique:  1.0
num 1-1 feats after filt:  48
1To1


people
Unique modA feats:  51
Unique modB feats:  400
% unique:  1.0
num 1-1 feats after filt:  51
1To1


nature
Unique modA feats:  49
Unique modB feats:  304
% unique:  1.0
num 1-1 feats after filt:  49
1To1


science
Unique modA feats:  56
Unique modB feats:  879
% unique:  1.0
num 1-1 feats after filt:  56
1To1


animals
Unique modA feats:  43
Unique modB feats:  137
% unique:  1.0
num 1-1 feats after filt:  43
1To1


Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  268
Unique modB feats:  417
% unique:  0.950354609929078
num 1-1 feats after filt:  268
1To1


people
Unique modA feats:  292
Unique modB feats:  400
% unique:  0.9931972789115646
num 1-1 feats after filt:  292
1To1


nature
Unique modA feats:  208
Unique modB feats:  304
% unique:  0.9952153110047847
num 1-1 feats after filt:  208
1To1


science
Unique modA feats:  553
Unique modB feats:  879
% unique:  0.9981949458483754
num 1-1 feats after filt:  553
1To1


animals
Unique modA feats:  107
Unique modB feats:  137
% unique:  1.0
num 1-1 feats after filt:  107
1To1


Layer: 2


...

numeric
Unique modA feats:  239
Unique modB feats:  417
% unique:  0.9795081967213115
num 1-1 feats after filt:  239
1To1


people
Unique modA feats:  273
Unique modB feats:  400
% unique:  0.9963503649635036
num 1-1 feats after filt:  273
1To1


nature
Unique modA feats:  198
Unique modB feats:  304
% unique:  1.0
num 1-1 feats after filt:  198
1To1


science
Unique modA feats:  514
Unique modB feats:  879
% unique:  0.9961240310077519
num 1-1 feats after filt:  514
1To1


animals
Unique modA feats:  98
Unique modB feats:  137
% unique:  1.0
num 1-1 feats after filt:  98
1To1


Layer: 4


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  245
Unique modB feats:  417
% unique:  0.9879032258064516
num 1-1 feats after filt:  245
1To1


people
Unique modA feats:  270
Unique modB feats:  400
% unique:  0.9926470588235294
num 1-1 feats after filt:  270
1To1


nature
Unique modA feats:  213
Unique modB feats:  304
% unique:  0.9906976744186047
num 1-1 feats after filt:  213
1To1


science
Unique modA feats:  500
Unique modB feats:  879
% unique:  0.9900990099009901
num 1-1 feats after filt:  500
1To1


animals
Unique modA feats:  104
Unique modB feats:  137
% unique:  1.0
num 1-1 feats after filt:  104
1To1


Layer: 5


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  263
Unique modB feats:  417
% unique:  0.9633699633699634
num 1-1 feats after filt:  263
1To1


people
Unique modA feats:  277
Unique modB feats:  400
% unique:  1.0
num 1-1 feats after filt:  277
1To1


nature
Unique modA feats:  214
Unique modB feats:  304
% unique:  1.0
num 1-1 feats after filt:  214
1To1


science
Unique modA feats:  515
Unique modB feats:  879
% unique:  1.0
num 1-1 feats after filt:  515
1To1


animals
Unique modA feats:  104
Unique modB feats:  137
% unique:  1.0
num 1-1 feats after filt:  104
1To1




In [90]:
with open(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L0

In [91]:
all_scores_dict = layer_to_dictscores[0]

In [92]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,43
nature,49
numeric,48
people,51
science,56


In [93]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.07,0.14,0.71,0.11,0.60
people,0.19,0.14,0.29,0.10,0.17
nature,0.13,0.12,0.41,0.13,0.49
science,0.22,0.10,0.08,0.10,0.08
animals,0.23,0.14,0.22,0.13,0.19


In [94]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,-0.02,-0.00,0.71,-0.00,0.78
people,0.03,-0.01,0.14,-0.01,0.16
nature,-0.02,-0.00,0.76,-0.01,0.70
science,-0.01,-0.00,0.72,0.00,0.71
animals,-0.05,-0.00,0.91,0.00,0.94


## analyze data: L1

In [95]:
all_scores_dict = layer_to_dictscores[1]

In [96]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,107
nature,208
numeric,268
people,292
science,553


In [97]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.676821
nature,0.652342
numeric,0.649026
people,0.619530
science,0.609709


In [98]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.11,0.10,0.38,0.05,0.05
people,0.34,0.10,0.00,0.05,0.00
nature,0.29,0.11,0.00,0.06,0.00
science,0.30,0.10,0.00,0.04,0.00
animals,0.01,0.10,0.92,0.07,0.86


In [99]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.03,0.03,0.34,-0.0,0.0
people,0.05,0.03,0.00,-0.0,0.0
nature,0.06,0.03,0.00,0.0,0.0
science,0.04,0.03,0.01,-0.0,0.0
animals,0.09,0.03,0.00,0.0,0.0


## analyze data: L2

In [100]:
all_scores_dict = layer_to_dictscores[2]

In [101]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,110
nature,213
numeric,277
people,283
science,555


In [102]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.648718
nature,0.614758
numeric,0.617613
people,0.593282
science,0.588367


In [103]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.28,0.11,0.05,0.05,0.00
people,0.34,0.10,0.01,0.05,0.00
nature,0.30,0.10,0.03,0.06,0.00
science,0.31,0.09,0.00,0.03,0.00
animals,0.13,0.12,0.37,0.07,0.11


In [104]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.05,0.04,0.02,0.0,0.0
people,0.06,0.04,0.01,0.0,0.0
nature,0.04,0.03,0.30,0.0,0.0
science,0.04,0.03,0.16,-0.0,0.0
animals,0.12,0.04,0.00,0.0,0.0


## analyze data: L3

In [105]:
all_scores_dict = layer_to_dictscores[3]

In [106]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,98
nature,198
numeric,239
people,273
science,514


In [107]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.580160
nature,0.552301
numeric,0.571575
people,0.533528
science,0.544166


In [108]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.37,0.05,0.00,0.05,0.0
people,0.32,0.06,0.00,0.04,0.0
nature,0.35,0.07,0.00,0.06,0.0
science,0.27,0.05,0.00,0.03,0.0
animals,0.01,0.09,0.94,0.08,0.9


In [109]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.04,0.03,0.06,-0.0,0.0
people,0.06,0.03,0.00,-0.0,0.0
nature,0.06,0.03,0.01,0.0,0.0
science,0.04,0.03,0.05,-0.0,0.0
animals,0.08,0.03,0.00,-0.0,0.0


## analyze data: L4

In [110]:
all_scores_dict = layer_to_dictscores[4]

In [111]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,104
nature,213
numeric,245
people,270
science,500


In [112]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.233008
nature,0.242170
numeric,0.331115
people,0.242762
science,0.281578


In [113]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.01,0.08,0.90,0.05,0.83
people,0.03,0.08,0.74,0.05,0.60
nature,0.03,0.07,0.81,0.05,0.56
science,0.11,0.06,0.10,0.04,0.03
animals,0.15,0.08,0.11,0.08,0.15


In [114]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.01,-0.0,0.00
people,0.01,0.01,0.71,-0.0,0.11
nature,0.00,0.01,0.85,-0.0,0.39
science,0.02,0.01,0.02,0.0,0.00
animals,0.03,0.01,0.12,-0.0,0.04


## analyze data: L5

In [115]:
all_scores_dict = layer_to_dictscores[4]

In [116]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,104
nature,213
numeric,245
people,270
science,500


In [117]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.233008
nature,0.242170
numeric,0.331115
people,0.242762
science,0.281578


In [118]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.01,0.08,0.90,0.05,0.83
people,0.03,0.08,0.74,0.05,0.60
nature,0.03,0.07,0.81,0.05,0.56
science,0.11,0.06,0.10,0.04,0.03
animals,0.15,0.08,0.11,0.08,0.15


In [119]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.01,-0.0,0.00
people,0.01,0.01,0.71,-0.0,0.11
nature,0.00,0.01,0.85,-0.0,0.39
science,0.02,0.01,0.02,0.0,0.00
animals,0.03,0.01,0.12,-0.0,0.04


# loop for any layer pair: L6

In [120]:
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
layer_id_2 = 6

hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [121]:
# store feature : lst of top strs
fList_model_B = []
samp_m = 5

for feature_idx in range(feature_acts_B.shape[-1]):
    if feature_idx % 5000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  5000
Feature:  10000
Feature:  15000
Feature:  20000
Feature:  25000
Feature:  30000


In [122]:
import pickle
from google.colab import files
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(fList_model_B, f)
files.download(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [123]:
# with open('fList_model_B.pkl', 'rb') as f:
#     fList_model_B = pickle.load(f)

In [124]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B, outputs)

Layer: 0


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  58
Unique modB feats:  733
% unique:  1.0
num 1-1 feats after filt:  58
1To1


people
Unique modA feats:  51
Unique modB feats:  306
% unique:  1.0
num 1-1 feats after filt:  51
1To1


nature
Unique modA feats:  51
Unique modB feats:  320
% unique:  1.0
num 1-1 feats after filt:  51
1To1


science
Unique modA feats:  59
Unique modB feats:  1679
% unique:  1.0
num 1-1 feats after filt:  59
1To1


animals
Unique modA feats:  32
Unique modB feats:  83
% unique:  1.0
num 1-1 feats after filt:  32
1To1


Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  369
Unique modB feats:  733
% unique:  0.9634464751958225
num 1-1 feats after filt:  369
1To1


people
Unique modA feats:  207
Unique modB feats:  306
% unique:  0.9672897196261683
num 1-1 feats after filt:  207
1To1


nature
Unique modA feats:  226
Unique modB feats:  320
% unique:  0.9955947136563876
num 1-1 feats after filt:  226
1To1


science
Unique modA feats:  647
Unique modB feats:  1679
% unique:  0.9953846153846154
num 1-1 feats after filt:  647
1To1


animals
Unique modA feats:  67
Unique modB feats:  83
% unique:  1.0
num 1-1 feats after filt:  67
1To1


Layer: 2


...

numeric
Unique modA feats:  390
Unique modB feats:  733
% unique:  0.9701492537313433
num 1-1 feats after filt:  390
1To1


people
Unique modA feats:  202
Unique modB feats:  306
% unique:  0.9805825242718447
num 1-1 feats after filt:  202
1To1


nature
Unique modA feats:  228
Unique modB feats:  320
% unique:  0.9956331877729258
num 1-1 feats after filt:  228
1To1


science
Unique modA feats:  620
Unique modB feats:  1679
% unique:  0.9904153354632588
num 1-1 feats after filt:  620
1To1


animals
Unique modA feats:  64
Unique modB feats:  83
% unique:  1.0
num 1-1 feats after filt:  64
1To1


Layer: 4


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  451
Unique modB feats:  733
% unique:  0.9636752136752137
num 1-1 feats after filt:  451
1To1


people
Unique modA feats:  208
Unique modB feats:  306
% unique:  0.985781990521327
num 1-1 feats after filt:  208
1To1


nature
Unique modA feats:  244
Unique modB feats:  320
% unique:  1.0
num 1-1 feats after filt:  244
1To1


science
Unique modA feats:  796
Unique modB feats:  1679
% unique:  0.9888198757763975
num 1-1 feats after filt:  796
1To1


animals
Unique modA feats:  68
Unique modB feats:  83
% unique:  1.0
num 1-1 feats after filt:  68
1To1


Layer: 5


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  443
Unique modB feats:  733
% unique:  0.9693654266958425
num 1-1 feats after filt:  443
1To1


people
Unique modA feats:  241
Unique modB feats:  306
% unique:  0.9757085020242915
num 1-1 feats after filt:  241
1To1


nature
Unique modA feats:  250
Unique modB feats:  320
% unique:  0.9920634920634921
num 1-1 feats after filt:  250
1To1


science
Unique modA feats:  778
Unique modB feats:  1679
% unique:  0.9910828025477707
num 1-1 feats after filt:  778
1To1


animals
Unique modA feats:  77
Unique modB feats:  83
% unique:  1.0
num 1-1 feats after filt:  77
1To1




In [125]:
with open(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L0

In [126]:
all_scores_dict = layer_to_dictscores[0]

In [127]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,32
nature,51
numeric,58
people,51
science,59


In [128]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.07,0.11,0.64,0.09,0.52
people,0.12,0.12,0.42,0.12,0.38
nature,0.13,0.12,0.39,0.12,0.37
science,0.14,0.13,0.32,0.12,0.36
animals,0.07,0.17,0.76,0.13,0.61


In [129]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.04,0.01,0.17,-0.00,0.08
people,0.02,0.00,0.21,-0.01,0.16
nature,-0.03,-0.00,0.80,0.00,0.76
science,0.02,0.00,0.27,-0.00,0.16
animals,0.04,0.01,0.29,0.01,0.27


## analyze data: L1

In [130]:
all_scores_dict = layer_to_dictscores[1]

In [131]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,67
nature,226
numeric,369
people,207
science,647


In [132]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.413574
nature,0.382295
numeric,0.335632
people,0.367423
science,0.396192


In [133]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.17,0.05,0.00,0.04,0.00
people,0.36,0.06,0.00,0.06,0.00
nature,0.36,0.06,0.00,0.05,0.00
science,0.31,0.03,0.00,0.03,0.00
animals,0.21,0.12,0.18,0.10,0.07


In [134]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.02,0.59,-0.0,0.0
people,0.08,0.02,0.00,-0.0,0.0
nature,0.12,0.02,0.00,0.0,0.0
science,0.05,0.02,0.00,-0.0,0.0
animals,0.08,0.02,0.03,-0.0,0.0


## analyze data: L2

In [135]:
all_scores_dict = layer_to_dictscores[2]

In [136]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,69
nature,228
numeric,430
people,209
science,754


In [137]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.419951
nature,0.395693
numeric,0.389239
people,0.391523
science,0.432983


In [138]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.03,0.07,0.69,0.04,0.55
people,0.45,0.08,0.00,0.06,0.00
nature,0.38,0.08,0.00,0.04,0.00
science,0.34,0.06,0.00,0.03,0.00
animals,0.48,0.11,0.00,0.09,0.00


In [139]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.03,0.02,0.15,-0.0,0.0
people,0.08,0.02,0.00,0.0,0.0
nature,0.11,0.02,0.00,-0.0,0.0
science,0.05,0.02,0.00,0.0,0.0
animals,0.13,0.02,0.00,0.0,0.0


## analyze data: L3

In [140]:
all_scores_dict = layer_to_dictscores[3]

In [141]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,64
nature,228
numeric,390
people,202
science,620


In [142]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.407217
nature,0.364089
numeric,0.343812
people,0.372243
science,0.395479


In [143]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.20,0.07,0.0,0.04,0.0
people,0.42,0.08,0.0,0.05,0.0
nature,0.55,0.08,0.0,0.05,0.0
science,0.38,0.07,0.0,0.03,0.0
animals,0.60,0.12,0.0,0.09,0.0


In [144]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.05,0.04,0.08,-0.0,0.0
people,0.08,0.04,0.00,0.0,0.0
nature,0.17,0.04,0.00,0.0,0.0
science,0.07,0.04,0.00,0.0,0.0
animals,0.17,0.04,0.00,0.0,0.0


## analyze data: L4

In [145]:
all_scores_dict = layer_to_dictscores[4]

In [146]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,68
nature,244
numeric,451
people,208
science,796


In [147]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.245290
nature,0.262698
numeric,0.361988
people,0.294102
science,0.396921


In [148]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.00,0.06,0.98,0.03,0.90
people,0.03,0.08,0.84,0.06,0.67
nature,0.15,0.07,0.04,0.05,0.02
science,0.08,0.06,0.23,0.03,0.01
animals,0.41,0.11,0.00,0.10,0.00


In [149]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.04,-0.0,0.00
people,0.02,0.01,0.32,-0.0,0.03
nature,0.06,0.01,0.00,0.0,0.00
science,0.02,0.01,0.00,0.0,0.00
animals,0.05,0.01,0.09,0.0,0.03


## analyze data: L5

In [150]:
all_scores_dict = layer_to_dictscores[4]

In [151]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,68
nature,244
numeric,451
people,208
science,796


In [152]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.245290
nature,0.262698
numeric,0.361988
people,0.294102
science,0.396921


In [153]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.00,0.06,0.98,0.03,0.90
people,0.03,0.08,0.84,0.06,0.67
nature,0.15,0.07,0.04,0.05,0.02
science,0.08,0.06,0.23,0.03,0.01
animals,0.41,0.11,0.00,0.10,0.00


In [154]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.04,-0.0,0.00
people,0.02,0.01,0.32,-0.0,0.03
nature,0.06,0.01,0.00,0.0,0.00
science,0.02,0.01,0.00,0.0,0.00
animals,0.05,0.01,0.09,0.0,0.03


# loop for any layer pair: L11

In [160]:
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
layer_id_2 = 11

hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [161]:
# store feature : lst of top strs
fList_model_B = []
samp_m = 5

for feature_idx in range(feature_acts_B.shape[-1]):
    if feature_idx % 5000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  5000
Feature:  10000
Feature:  15000
Feature:  20000
Feature:  25000
Feature:  30000


In [162]:
import pickle
from google.colab import files
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(fList_model_B, f)
files.download(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [163]:
# with open('fList_model_B.pkl', 'rb') as f:
#     fList_model_B = pickle.load(f)

In [164]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B, outputs)

Layer: 0


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  53
Unique modB feats:  479
% unique:  1.0
num 1-1 feats after filt:  53
1To1


people
Unique modA feats:  46
Unique modB feats:  185
% unique:  1.0
num 1-1 feats after filt:  46
1To1


nature
Unique modA feats:  40
Unique modB feats:  204
% unique:  1.0
num 1-1 feats after filt:  40
1To1


science
Unique modA feats:  58
Unique modB feats:  1436
% unique:  1.0
num 1-1 feats after filt:  58
1To1


animals
Unique modA feats:  28
Unique modB feats:  52
% unique:  1.0
num 1-1 feats after filt:  28
1To1


Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  277
Unique modB feats:  479
% unique:  0.9551724137931035
num 1-1 feats after filt:  277
1To1


people
Unique modA feats:  128
Unique modB feats:  185
% unique:  0.9770992366412213
num 1-1 feats after filt:  128
1To1


nature
Unique modA feats:  154
Unique modB feats:  204
% unique:  0.9685534591194969
num 1-1 feats after filt:  154
1To1


science
Unique modA feats:  393
Unique modB feats:  1436
% unique:  0.9899244332493703
num 1-1 feats after filt:  393
1To1


animals
Unique modA feats:  44
Unique modB feats:  52
% unique:  1.0
num 1-1 feats after filt:  44
1To1


Layer: 2


...

numeric
Unique modA feats:  269
Unique modB feats:  479
% unique:  0.9572953736654805
num 1-1 feats after filt:  269
1To1


people
Unique modA feats:  141
Unique modB feats:  185
% unique:  0.9724137931034482
num 1-1 feats after filt:  141
1To1


nature
Unique modA feats:  158
Unique modB feats:  204
% unique:  0.9575757575757575
num 1-1 feats after filt:  158
1To1


science
Unique modA feats:  362
Unique modB feats:  1436
% unique:  0.9863760217983651
num 1-1 feats after filt:  362
1To1


animals
Unique modA feats:  45
Unique modB feats:  52
% unique:  1.0
num 1-1 feats after filt:  45
1To1


Layer: 4


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  320
Unique modB feats:  479
% unique:  0.9696969696969697
num 1-1 feats after filt:  320
1To1


people
Unique modA feats:  146
Unique modB feats:  185
% unique:  0.9733333333333334
num 1-1 feats after filt:  146
1To1


nature
Unique modA feats:  166
Unique modB feats:  204
% unique:  0.9651162790697675
num 1-1 feats after filt:  166
1To1


science
Unique modA feats:  567
Unique modB feats:  1436
% unique:  0.984375
num 1-1 feats after filt:  567
1To1


animals
Unique modA feats:  48
Unique modB feats:  52
% unique:  1.0
num 1-1 feats after filt:  48
1To1


Layer: 5


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

numeric
Unique modA feats:  327
Unique modB feats:  479
% unique:  0.9646017699115044
num 1-1 feats after filt:  327
1To1


people
Unique modA feats:  153
Unique modB feats:  185
% unique:  0.9745222929936306
num 1-1 feats after filt:  153
1To1


nature
Unique modA feats:  171
Unique modB feats:  204
% unique:  0.9941860465116279
num 1-1 feats after filt:  171
1To1


science
Unique modA feats:  532
Unique modB feats:  1436
% unique:  0.9888475836431226
num 1-1 feats after filt:  532
1To1


animals
Unique modA feats:  47
Unique modB feats:  52
% unique:  1.0
num 1-1 feats after filt:  47
1To1




In [165]:
with open(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L0

In [166]:
all_scores_dict = layer_to_dictscores[0]

In [167]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,28
nature,40
numeric,53
people,46
science,58


In [168]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.06,0.12,0.63,0.12,0.64
people,0.01,0.12,0.88,0.13,0.93
nature,0.03,0.11,0.79,0.14,0.86
science,0.00,0.11,0.97,0.10,0.98
animals,0.06,0.13,0.74,0.14,0.73


In [169]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.00,0.0,0.51,-0.0,0.39
people,0.04,0.0,0.17,-0.0,0.09
nature,-0.00,0.0,0.56,-0.0,0.51
science,-0.01,-0.0,0.63,-0.0,0.60
animals,0.10,0.0,0.05,-0.0,0.03


## analyze data: L1

In [170]:
all_scores_dict = layer_to_dictscores[1]

In [171]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,44
nature,154
numeric,277
people,128
science,393


In [172]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.467776
nature,0.344696
numeric,0.256175
people,0.317198
science,0.339722


In [173]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.14,0.05,0.02,0.05,0.03
people,0.12,0.07,0.17,0.07,0.19
nature,0.37,0.06,0.00,0.06,0.00
science,0.13,0.05,0.02,0.04,0.01
animals,0.01,0.13,0.98,0.12,0.98


In [174]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.01,0.01,0.48,0.00,0.01
people,0.06,0.01,0.00,-0.00,0.00
nature,0.06,0.01,0.00,0.00,0.00
science,0.03,0.01,0.00,0.00,0.00
animals,0.01,0.01,0.51,0.01,0.43


## analyze data: L2

In [175]:
all_scores_dict = layer_to_dictscores[2]

In [176]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,46
nature,160
numeric,276
people,140
science,384


In [177]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.466532
nature,0.347998
numeric,0.284955
people,0.346401
science,0.387382


In [178]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.29,0.05,0.00,0.05,0.00
people,0.49,0.06,0.00,0.06,0.00
nature,0.34,0.07,0.00,0.07,0.00
science,0.29,0.04,0.00,0.04,0.00
animals,0.26,0.13,0.06,0.12,0.08


In [179]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.03,-0.0,0.00
people,0.04,0.01,0.00,0.0,0.00
nature,0.06,0.01,0.00,0.0,0.00
science,0.04,0.01,0.00,-0.0,0.00
animals,0.07,0.00,0.01,-0.0,0.01


## analyze data: L3

In [180]:
all_scores_dict = layer_to_dictscores[3]

In [181]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,45
nature,158
numeric,269
people,141
science,362


In [182]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.469878
nature,0.341629
numeric,0.273476
people,0.332259
science,0.349563


In [183]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.47,0.05,0.00,0.04,0.00
people,0.49,0.08,0.00,0.07,0.00
nature,0.61,0.07,0.00,0.07,0.00
science,0.31,0.04,0.00,0.04,0.00
animals,0.37,0.14,0.02,0.12,0.02


In [184]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.04,0.01,0.00,-0.0,0.0
people,0.07,0.01,0.00,0.0,0.0
nature,0.11,0.01,0.00,-0.0,0.0
science,0.04,0.01,0.00,0.0,0.0
animals,0.09,0.01,0.03,0.0,0.0


## analyze data: L4

In [185]:
all_scores_dict = layer_to_dictscores[4]

In [186]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,48
nature,166
numeric,320
people,146
science,567


In [187]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.280315
nature,0.218756
numeric,0.319094
people,0.297455
science,0.424121


In [188]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.09,0.05,0.12,0.05,0.11
people,0.06,0.07,0.52,0.06,0.48
nature,0.17,0.06,0.03,0.06,0.03
science,0.05,0.04,0.34,0.03,0.22
animals,0.16,0.11,0.28,0.13,0.37


In [189]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.11,0.0,0.00
people,0.06,0.01,0.00,-0.0,0.00
nature,0.04,0.01,0.00,0.0,0.00
science,0.03,0.01,0.00,0.0,0.00
animals,0.06,0.02,0.11,-0.0,0.02


## analyze data: L5

In [190]:
all_scores_dict = layer_to_dictscores[4]

In [191]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
animals,48
nature,166
numeric,320
people,146
science,567


In [192]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.280315
nature,0.218756
numeric,0.319094
people,0.297455
science,0.424121


In [193]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.09,0.05,0.12,0.05,0.11
people,0.06,0.07,0.52,0.06,0.48
nature,0.17,0.06,0.03,0.06,0.03
science,0.05,0.04,0.34,0.03,0.22
animals,0.16,0.11,0.28,0.13,0.37


In [194]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.11,0.0,0.00
people,0.06,0.01,0.00,-0.0,0.00
nature,0.04,0.01,0.00,0.0,0.00
science,0.03,0.01,0.00,0.0,0.00
animals,0.06,0.02,0.11,-0.0,0.02
